In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
np.set_printoptions(precision=3)

# Q1

In [2]:
# my_dict = {"user": ['A','A','A','A','A','A','B','B','B','B','B','B','C','C','C','C','C','C'], 
#            "item": ['a','b','d','e','g','h','b','c','d','e','f','g','a','c','d','f','g','h'], 
#            "rating": [5,4,4,2,4,1,5,5,4,2,1,2,1,1,5,5,3,4]}

my_dict = {"user": ['A','A','A','A','A','A','B','B','B','B','B','B','C','C','C','C','C','C'], 
           "item": ['a','b','d','e','g','h','b','c','d','e','f','g','a','c','d','f','g','h'], 
           "rating": [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]}

df = pd.DataFrame(my_dict)
df

,user,item,rating
0,A,a,1
1,A,b,1
2,A,d,1
3,A,e,1
4,A,g,1
5,A,h,1
6,B,b,1
7,B,c,1
8,B,d,1
9,B,e,1


In [3]:
df_table = df.pivot_table(index='user', columns='item', values='rating') 
df_matrix = df_table.fillna(0).to_numpy()
df_matrix

array([[1., 1., 0., 1., 1., 0., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 1., 1., 0., 1., 1., 1.]])

In [4]:
# symmetric 이다
cosine_similarity(df_matrix)

array([[1.   , 0.667, 0.667],
       [0.667, 1.   , 0.667],
       [0.667, 0.667, 1.   ]])

In [5]:
my_dict = {"user": ['A','A','A','A','A','A','B','B','B','B','B','B','C','C','C','C','C','C'], 
           "item": ['a','b','d','e','g','h','b','c','d','e','f','g','a','c','d','f','g','h'], 
           "rating": [1,1,1,0,1,0,1,1,1,0,0,0,0,0,1,1,1,1]}

df = pd.DataFrame(my_dict)
df_table = df.pivot_table(index='user', columns='item', values='rating') 
df_matrix = df_table.fillna(0).to_numpy()
df_matrix

array([[1., 1., 0., 1., 0., 0., 1., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 1., 1.]])

In [6]:
cosine_similarity(df_matrix)

array([[1.   , 0.577, 0.5  ],
       [0.577, 1.   , 0.289],
       [0.5  , 0.289, 1.   ]])

(e)

https://stackoverflow.com/questions/21057434/an-efficient-way-to-calculate-the-mean-of-each-column-or-row-of-non-zero-element

In [18]:
my_dict = {"user": ['A','A','A','A','A','A','B','B','B','B','B','B','C','C','C','C','C','C'], 
           "item": ['a','b','d','e','g','h','b','c','d','e','f','g','a','c','d','f','g','h'], 
           "rating": [5,4,4,2,4,1,5,5,4,2,1,2,1,1,5,5,3,4]}
df = pd.DataFrame(my_dict)
df_table = df.pivot_table(index='user', columns='item', values='rating') 
df_matrix = df_table.fillna(0).to_numpy()
df_matrix

array([[5., 4., 0., 4., 2., 0., 4., 1.],
       [0., 5., 5., 4., 2., 1., 2., 0.],
       [1., 0., 1., 5., 0., 5., 3., 4.]])

In [8]:
# non empty entry에 대해서 평균을 구함
def get_means(ratings):
    ratings[np.where(ratings == 0)] = np.nan

    user_means = np.nanmean(ratings, axis=1)
    movie_means = np.nanmean(ratings, axis=0)

    return user_means, movie_means

In [9]:
user_means, movie_means = get_means(df_matrix)

In [10]:
print(user_means)
print(movie_means)

[3.333 3.167 3.167]
[3.    4.5   3.    4.333 2.    3.    3.    2.5  ]


In [11]:
user_means.shape

(3,)

In [12]:
k = user_means.reshape(3,-1)
k
# k = np.array([[1],[2],[3]]) 

array([[3.333],
       [3.167],
       [3.167]])

https://stackoverflow.com/questions/26243383/subtract-ith-value-of-a-vector-from-non-zero-values-of-the-ith-row-of-a-sparse-m

In [13]:
normalized = (df_matrix - k)*(df_matrix != 0)
print(normalized)

[[ 1.667  0.667    nan  0.667 -1.333    nan  0.667 -2.333]
 [   nan  1.833  1.833  0.833 -1.167 -2.167 -1.167    nan]
 [-2.167    nan -2.167  1.833    nan  1.833 -0.167  0.833]]


In [14]:
normalized = np.nan_to_num(normalized)
normalized

array([[ 1.667,  0.667,  0.   ,  0.667, -1.333,  0.   ,  0.667, -2.333],
       [ 0.   ,  1.833,  1.833,  0.833, -1.167, -2.167, -1.167,  0.   ],
       [-2.167,  0.   , -2.167,  1.833,  0.   ,  1.833, -0.167,  0.833]])

In [15]:
# user-user similarity
user_correlation = cosine_similarity(normalized)
user_correlation

array([[ 1.   ,  0.197, -0.322],
       [ 0.197,  1.   , -0.394],
       [-0.322, -0.394,  1.   ]])

In [25]:
df_matrix = np.nan_to_num(df_matrix)
df_matrix

array([[5., 4., 0., 4., 2., 0., 4., 1.],
       [0., 5., 5., 4., 2., 1., 2., 0.],
       [1., 0., 1., 5., 0., 5., 3., 4.]])

In [26]:
pred = user_correlation.dot(df_matrix)
pred

array([[ 4.678,  4.986,  0.664,  3.18 ,  2.394, -1.412,  3.429, -0.287],
       [ 0.592,  5.788,  4.606,  2.82 ,  2.394, -0.969,  1.607, -1.378],
       [-0.609, -3.256, -0.969,  2.138, -1.431,  4.606,  0.925,  3.678]])

In [32]:
my_dict = {"user": ['A','A','A','A','A','A','B','B','B','B','B','B','C','C','C','C','C','C'], 
           "item": ['a','b','d','e','g','h','b','c','d','e','f','g','a','c','d','f','g','h'], 
           "rating": [5,4,4,2,4,1,5,5,4,2,1,2,1,1,5,5,3,4]}
df = pd.DataFrame(my_dict)
df_table = df.pivot_table(index='user', columns='item', values='rating') 
# df_matrix = df_table.fillna(0).to_numpy()
df_matrix = df_table.to_numpy()
df_matrix

array([[ 5.,  4., nan,  4.,  2., nan,  4.,  1.],
       [nan,  5.,  5.,  4.,  2.,  1.,  2., nan],
       [ 1., nan,  1.,  5., nan,  5.,  3.,  4.]])

In [46]:
diff = (df_matrix - pred)*(~np.isnan(df_matrix))
diff

array([[ 0.322, -0.986,    nan,  0.82 , -0.394,    nan,  0.571,  1.287],
       [   nan, -0.788,  0.394,  1.18 , -0.394,  1.969,  0.393,    nan],
       [ 1.609,    nan,  1.969,  2.862,    nan,  0.394,  2.075,  0.322]])

In [50]:
# non empty entry 수 N
N = np.count_nonzero(~np.isnan(diff))
N

18

In [53]:
temp = np.nan_to_num(diff)**2/N
temp

array([[0.006, 0.054, 0.   , 0.037, 0.009, 0.   , 0.018, 0.092],
       [0.   , 0.035, 0.009, 0.077, 0.009, 0.215, 0.009, 0.   ],
       [0.144, 0.   , 0.215, 0.455, 0.   , 0.009, 0.239, 0.006]])

In [58]:
rmse = sqrt(temp.sum())
rmse

1.2793654783961728